In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import glob
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from CloudDetect.util import read_off, visualise
from CloudDetect.transform import PointCloudSample, Normalise, Tensor
from CloudDetect.models import PointNet
from CloudDetect.dataset import CloudDataset, assign_val_indices
from CloudDetect.util import get_metrics
from CloudDetect.train_validate import training

In [3]:
metadata = pd.read_csv('data/metadata_modelnet40.csv')
all_data_files = glob.glob('**/*.off',recursive=True)
all_data_files = set(['/'.join(x.split('/')[2:]) for x in all_data_files])
metadata = metadata[metadata['object_path'].map(lambda x: x in all_data_files)]

In [4]:
# data loader

In [5]:
def preprocessing(n_sample, norm_how):
    composition = [
            PointCloudSample(n_sample),
            Normalise(norm_how),
            Tensor()
    ]
    return transforms.Compose(composition)

def generate_class_mapper(metadata):
    class_mapper = {x:i for i,x in enumerate(metadata['class'].unique())}
    return class_mapper

In [6]:
metadata_train = metadata[metadata['split'] == 'train']
class_mapper = generate_class_mapper(metadata_train)
metadata_test = metadata[metadata['split'] == 'test']
metadata_train = metadata_train.reset_index(drop = True)
metadata_test = metadata_test.reset_index(drop = True)
assign_val_indices(metadata_train, 'class', 3)

In [7]:
# modelling
loss_fn = torch.nn.NLLLoss()
model = PointNet(n_point = 1024, classes = len(class_mapper), segment = False).to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=0.0001)

In [8]:
val_idx = 2
metadata_train_val = metadata_train[metadata_train['kfold'] == val_idx]
metadata_train_train = metadata_train[metadata_train['kfold'] != val_idx]

# jsut for quick training
#metadata_train_train = metadata_train_train.sample(100)
#metadata_train_val = metadata_train_val.sample(100)

# dataset
ROOT = 'data/ModelNet40/'
preprocessor = preprocessing(1024, 'max')
class_mapper = generate_class_mapper(metadata)

cloud_train_dataset = CloudDataset(metadata_train_train, preprocessor, ROOT, class_mapper, device)
cloud_val_dataset = CloudDataset(metadata_train_val, preprocessor, ROOT, class_mapper, device)

CloudDataTrainLoader = DataLoader(cloud_train_dataset, batch_size=32, shuffle=True)
CloudDataValLoader = DataLoader(cloud_val_dataset, batch_size=32, shuffle=False)

In [9]:
train_object = training(loss_fn, optimiser, model, CloudDataTrainLoader, CloudDataValLoader, 10, False, device)

In [ ]:
train_object.run(num_epoch = 2)

BATCH TRAIN LOSS -999 - VAL LOSS -999:   1%|    | 2/206 [00:07<11:46,  3.46s/it]

In [95]:
# dataset
ROOT = 'data/ModelNet40/'
preprocessor = preprocessing(1024, 'max')
class_mapper = generate_class_mapper(metadata)

cloud_train_dataset = CloudDataset(metadata_train_train, preprocessor, ROOT, class_mapper)
cloud_test_dataset = CloudDataset(metadata_test.sample(100), preprocessor, ROOT, class_mapper)

CloudDataTrainLoader = DataLoader(cloud_train_dataset, batch_size=32, shuffle=True)
CloudDataValLoader = DataLoader(cloud_test_dataset, batch_size=32, shuffle=False)

In [96]:
train_object = training(loss_fn, optimiser,model, CloudDataTrainLoader, CloudDataValLoader)

In [97]:
train_object.run(num_epoch = 2)

BATCH TRAIN LOSS 3.56587 - VAL LOSS 3.73725: 100%|█| 4/4 [00:40<00:00, 10.22s/it
/opt/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'f1_score': 0.017671517671517672, 'precision': 0.012012012012012012, 'recall': 0.033783783783783786, 'balanced_acc': 0.0390625, 'loss': 3.746208429336548}


BATCH TRAIN LOSS -999 - VAL LOSS -999:  25%|█▌    | 1/4 [00:06<00:20,  6.68s/it]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/opt/miniconda3/envs/main/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_24730/370393514.py", line 1, in <module>
    train_object.run(num_epoch = 2)
  File "/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_24730/1807974284.py", line 88, in run
    self.run_epoch(epoch_idx = i)
  File "/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_24730/1807974284.py", line 45, in run_epoch
    loss = self.perform_optimisation(X, y)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_24730/1807974284.py", line 22, in perform_optimisation
    model_output = self.model(X)
                   ^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/main/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_i